In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

model_name = "openai/gpt-oss-20b"

In [ ]:
def load_and_prepare_datasets():
    ds1 = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/health_train.jsonl"})["train"]
    ds2 = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/rehab_train.jsonl"})["train"]
    ds3 = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/security_train.jsonl"})["train"]
    combined = concatenate_datasets([ds1, ds2, ds3])
    return combined

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

In [ ]:
dataset = load_and_prepare_datasets()
tokenized_dataset = dataset.map(preprocess, batched=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/output",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    fp16=True,  # GPU 사용 시 True, CPU면 False
    save_strategy="epoch",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()